In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
dataset = pd.read_csv('Data.csv')
x = dataset.iloc[: , : -1].values
y = dataset.iloc[: ,  -1].values

In [22]:
dataset.nunique()

Country      3
Age          9
Salary       9
Purchased    2
dtype: int64

In [23]:
print(x)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [24]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


take care of messing data (nan)

In [25]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan , strategy='mean')
imputer.fit(x[:,1:3]) # analyzes the data you provide and calculates the necessary statistics
x[:,1:3] = imputer.transform(x[:,1:3]) # method uses the statistics computed during .fit() to replace the missing values

In [26]:
print(x)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


Encoding the 3 classes (France , spain , germany) into 3 column usong *OneHotEncoder*

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder' ,OneHotEncoder() ,[0] )] , remainder="passthrough")
x = np.array(ct.fit_transform(x))

In [28]:
print(x)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


change yes and no to 0 and 1

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [30]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


splait dataset into train and test

In [31]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x , y ,test_size=0.2 , random_state=1)

In [32]:
print(x_train)   

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [33]:
print(x_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [34]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [35]:
print(y_test)

[0 1]


# featur scaling

The difference between using `fit_transform` on `x_train` and `transform` on `x_test` when using `StandardScaler` lies in **why** and **when** the scaling parameters (mean and standard deviation) are computed and reused.

---

### **Why `fit_transform` for `x_train`?**
1. **`fit_transform` combines two steps:**
   - `fit`: Computes the mean and standard deviation of the data (needed to scale the data).
   - `transform`: Applies the scaling (subtracting the mean and dividing by the standard deviation).

2. **Purpose in `x_train`:**
   - The model should learn the scaling parameters (mean and standard deviation) **only from the training data** because it represents the real-world data distribution your model will encounter during training.
   - Applying scaling on the same data ensures consistency and prevents information leakage.

---

### **Why `transform` for `x_test`?**
1. **`transform` uses the scaling parameters** (mean and standard deviation) that were computed during the `fit` on `x_train`.
2. **Purpose in `x_test`:**
   - Test data should **not be used to calculate scaling parameters**, as the test set is supposed to simulate unseen data.
   - Scaling the test set with the parameters learned from the training set ensures that the model evaluates data under the same conditions it was trained on.

---

### **Key Difference:**
- `fit_transform`: **Fits** the scaler on data (computes mean & std) and then **transforms** it. Use this on the **training data**.
- `transform`: Reuses the previously computed parameters to scale new data (e.g., test data or new unseen data).

---

### **Code Context Example:**
```python
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Example data
import numpy as np
data = np.array([[1, 2, 3, 100], 
                 [4, 5, 6, 200], 
                 [7, 8, 9, 300], 
                 [10, 11, 12, 400]])

# Splitting data
x_train, x_test = train_test_split(data, test_size=0.5, random_state=42)

# Applying StandardScaler
scaler = StandardScaler()

# Fit and transform training data
x_train_scaled = scaler.fit_transform(x_train[:, 3:])  # Scale only the 4th column (index 3)

# Transform test data using the training parameters
x_test_scaled = scaler.transform(x_test[:, 3:])
```

Here:
- `fit_transform` is used for `x_train` to compute scaling parameters (mean, std) from the training data.
- `transform` is used for `x_test` to apply the same scaling based on those parameters, ensuring consistency.

Let me know if you’d like more examples or explanations!

In [36]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train=sc.fit_transform(x_train[: , 3:])
x_test = sc.transform(x_test[:,3:])

In [37]:
print(x_train)

[[-0.19159184 -1.07812594]
 [-0.01411729 -0.07013168]
 [ 0.56670851  0.63356243]
 [-0.30453019 -0.30786617]
 [-1.90180114 -1.42046362]
 [ 1.14753431  1.23265336]
 [ 1.43794721  1.57499104]
 [-0.74014954 -0.56461943]]


In [38]:
print(x_test)

[[-1.46618179 -0.9069571 ]
 [-0.44973664  0.20564034]]


: 